In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import TensorDataset, DataLoader

In [ ]:
years = list(range(2018, 2024))
dfs = []

# Update this to your actual file path
base_path = '/content/drive/MyDrive/CS412/final/stats/player_data_{}_final.csv'

for year in years:
    df = pd.read_csv(base_path.format(year))
    df['Year'] = year
    dfs.append(df)

train_df = pd.concat(dfs, ignore_index=True)
print(f"Combined training shape: {train_df.shape}")
train_df.head()


Combined training shape: (140, 12)


,Player,HLTV_ID,Rating,DPR,KAST,Impact,ADR,KPR,Rank,MVPs,EVPs,Year
0,s1mple,7998,1.35,0.59,76.4,1.42,87.2,0.88,1,6,4,2018
1,device,7592,1.23,0.60,72.5,1.31,81.4,0.79,2,7,1,2018
2,NiKo,3741,1.22,0.64,72.1,1.27,84.8,0.80,3,2,7,2018
3,electronic,8918,1.21,0.63,74.5,1.18,84.5,0.77,4,0,7,2018
4,dupreeh,7398,1.17,0.63,73.4,1.20,79.8,0.74,5,1,8,2018


In [17]:
test_df = pd.read_csv('/content/drive/MyDrive/CS412/final/stats/player_data_2024_final.csv')
test_df['Year'] = 2024


In [20]:
features = ['Rating', 'DPR', 'KAST', 'Impact', 'ADR', 'KPR', 'MVPs', 'EVPs']
target = 'Rank'

# Assign performance scores
train_df['Score'] = 101 - train_df['Rank']
test_df['Score'] = 0  # Placeholder

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(train_df[features])
y_train = train_df['Score'].values

X_test = scaler.transform(test_df[features])

from sklearn.preprocessing import PolynomialFeatures

# Add degree-2 interactions (cross-terms)
poly = PolynomialFeatures(degree=2, include_bias=False, interaction_only=False)
X_train_poly = poly.fit_transform(train_df[features])
X_test_poly = poly.transform(test_df[features])

# Scale after polynomial transformation
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_poly)
X_test_scaled = scaler.transform(X_test_poly)

# Target
y_train = train_df['Score'].values


In [21]:
class PlayerRankingNN_Deep(nn.Module):
    def __init__(self, input_dim):
        super(PlayerRankingNN_Deep, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.3),

            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Dropout(0.3),

            nn.Linear(64, 32),
            nn.ReLU(),

            nn.Linear(32, 16),
            nn.ReLU(),

            nn.Linear(16, 1)
        )

    def forward(self, x):
        return self.model(x)


Training Loop

In [22]:
from sklearn.model_selection import KFold

X_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
y_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)

kf = KFold(n_splits=5, shuffle=True, random_state=42)
cv_scores = []

for fold, (train_idx, val_idx) in enumerate(kf.split(X_tensor)):
    print(f"Fold {fold+1}/5")

    # Split
    X_train_fold = X_tensor[train_idx]
    y_train_fold = y_tensor[train_idx]
    X_val_fold = X_tensor[val_idx]
    y_val_fold = y_tensor[val_idx]

    # Datasets
    train_ds = TensorDataset(X_train_fold, y_train_fold)
    train_loader = DataLoader(train_ds, batch_size=8, shuffle=True)

    # Model
    model = PlayerRankingNN_Deep(X_train_scaled.shape[1])
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.MSELoss()

    # Training
    for epoch in range(50):
        model.train()
        for xb, yb in train_loader:
            optimizer.zero_grad()
            preds = model(xb)
            loss = criterion(preds, yb)
            loss.backward()
            optimizer.step()

    # Evaluate
    model.eval()
    with torch.no_grad():
        preds_val = model(X_val_fold)
        val_loss = criterion(preds_val, y_val_fold).item()
        cv_scores.append(val_loss)
        print(f"Fold {fold+1} Validation Loss: {val_loss:.4f}")


Fold 1/5
Fold 1 Validation Loss: 116.1720
Fold 2/5
Fold 2 Validation Loss: 171.7807
Fold 3/5
Fold 3 Validation Loss: 164.6689
Fold 4/5
Fold 4 Validation Loss: 152.2061
Fold 5/5
Fold 5 Validation Loss: 88.1599


In [23]:
print(f"Average CV Loss (MSE): {np.mean(cv_scores):.4f}")


Average CV Loss (MSE): 138.5975


In [25]:
# Final model training
model = PlayerRankingNN_Deep(X_train_scaled.shape[1])
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()

X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)

train_ds = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_ds, batch_size=8, shuffle=True)

for epoch in range(100):
    model.train()
    for xb, yb in train_loader:
        optimizer.zero_grad()
        preds = model(xb)
        loss = criterion(preds, yb)
        loss.backward()
        optimizer.step()
    if epoch % 10 == 0:
        print(f"Epoch {epoch}: Loss = {loss.item():.4f}")


Epoch 0: Loss = 8249.9619
Epoch 10: Loss = 145.5329
Epoch 20: Loss = 324.0816
Epoch 30: Loss = 81.9918
Epoch 40: Loss = 54.2317
Epoch 50: Loss = 69.6966
Epoch 60: Loss = 140.9562
Epoch 70: Loss = 75.2563
Epoch 80: Loss = 335.8057
Epoch 90: Loss = 13.9620


In [29]:
model.eval()
X_2024_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
with torch.no_grad():
    preds_2024 = model(X_2024_tensor).numpy()

test_df['PredictedScore'] = preds_2024
test_df_sorted = test_df.sort_values(by='PredictedScore', ascending=False)
top_20_2024 = test_df_sorted.head(20)
top_20_2024.reset_index(inplace=True)
print(top_20_2024[['Player', 'PredictedScore']])


      Player  PredictedScore
0       donk      142.939041
1      ZywOo      111.381645
2    malbsMd       95.783844
3      EliGE       92.404518
4   XANTARES       88.839867
5     m0NESY       86.120178
6        b1t       82.122017
7      Senzu       81.967453
8      sh1ro       81.681107
9    Twistzz       81.494415
10      NiKo       79.784225
11        jL       79.500542
12  KSCERATO       77.842636
13    frozen       77.811752
14     NertZ       77.676727
15       NAF       76.885056
16     Spinx       75.839844
17        iM       73.488480
18    device       73.262146
19      ropz       72.620926
